In [4]:
import pandas as pd
import os

In [5]:
PATH = '../data'

In [ ]:
def get_shape():
    for file in os.listdir(PATH):
        if file.endswith(".csv"):
            csv_file = os.path.join(PATH, file)
            df = pd.read_csv(csv_file)
            print(f"{csv_file} | Shape: {df.shape}") 

### BiLSTM

In [ ]:
get_shape()

### Attention based Encoder